# Branch and Bound

__author__ = "Rahul Kakodkar"
__copyright__ = "Me"
__credits__ = "Me"
__license__ = "Open"
__version__ = "0.0.7"
__maintainer__ = "Rahul Kakodkar"
__email__ = "Me"
__status__ = "Single"

**Import Modules**

In [2]:
from itertools import product
from pyomo.environ import *
import numpy 
import pandas 
from pyomo.opt import SolverStatus, TerminationCondition

Consider this problem from : http://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf


**Shop Floor Expansion with branch and bound**

The owner of a machining shop wants to increase revenue, but he has limited floor space (200 sq. Ft.) and money (40,000$)

In [7]:
pandas.DataFrame(data = {'Machine': ['Price', 'Lathe'], 'Floor Space (sq. ft.)': [15, 30], 'Purchase Price ($)': [8000, 4000], 'Revenue': [100, 150]})

,Machine,Floor Space (sq. ft.),Purchase Price ($),Revenue
0,Price,15,8000,100
1,Lathe,30,4000,150



*maximize* 100 $X_1$ + 150 $X_2$

8000 $X_1$ + 4000 $X_2$ $\leq$ 40000

15 $X_1$ + 30 $X_2$ $\leq$ 200

**Declare a general model**

In [78]:
def declare():
    model = ConcreteModel()

    model.X1 = Var(within = NonNegativeReals)
    model.X2 = Var(within = NonNegativeReals)

    model.money_cons = Constraint(expr = 8000*model.X1 + 4000*model.X2 <= 40000)
    model.area_cons = Constraint(expr = 15*model.X1 + 30*model.X2 <= 200)

    model.obj = Objective(expr = 100*model.X1 + 150*model.X2, sense = maximize)

    return model


**Printer function for important results**

In [9]:
def print_res(model):
    print(f"objective value is {model.obj()}, X1 = {model.X1.value}, X2 = {model.X2.value}")
    print(f"The lower bounds of variables are X1 = {floor(model.X1.value)}, X2 = {floor(model.X2.value)}")
    print(f"at the lower bounds the value of the objective is {100*floor(model.X1.value) + 150*floor(model.X2.value)}")
    print(f"The distance from the values are delX1 = {model.X1.value - floor(model.X1.value)}, delX2 = {model.X2.value - floor(model.X2.value)}")


**Node1**

Here the original IP is relaxed to get a LP

In [80]:
node1 = declare()
result = SolverFactory('gurobi').solve(node1)
print_res(node1)

objective value is 1055.5555555555554, X1 = 2.2222222222222223, X2 = 5.555555555555555
The lower bounds of variables are X1 = 2, X2 = 5
at the lower bounds the value of the objective is 950
The distance from the values are delX1 = 0.22222222222222232, delX2 = 0.5555555555555554


since $X_2$ has a larger delta, we branch at $X_2$

**Node2**

Add the constraint

$X_2 \leq 5$

In [81]:
node2 = declare()
node2.cons_add = Constraint(expr = node2.X2 <= 5)
result = SolverFactory('gurobi').solve(node2)
print_res(node2)

objective value is 1000.0, X1 = 2.5, X2 = 5.0
The lower bounds of variables are X1 = 2, X2 = 5
at the lower bounds the value of the objective is 950
The distance from the values are delX1 = 0.5, delX2 = 0.0


**Node3**

Add the constraint 

 $X_2 \geq 6$

In [91]:
node3 = declare()
node3.cons_add = Constraint(expr = node3.X2 >= 6)
result = SolverFactory('gurobi').solve(node3)
print_res(node3)


objective value is 1033.3333333333333, X1 = 1.3333333333333333, X2 = 6.0
The lower bounds of variables are X1 = 1, X2 = 6
at the lower bounds the value of the objective is 1000
The distance from the values are delX1 = 0.33333333333333326, delX2 = 0.0


Node 3 provides a better solution, let us branch out of node 3

**Node 4**

Retain the constraint from node 3

$X_2 >= 6$

Add the constraint

$X_1 <=1$

In [93]:
node4 = declare()
node4.cons_fix = Constraint(expr = node4.X2 >= 6)
node4.cons_add = Constraint(expr = node4.X1 <= 1)
result = SolverFactory('gurobi').solve(node4)
print_res(node4)

objective value is 1025.0, X1 = 1.0, X2 = 6.166666666666667
The lower bounds of variables are X1 = 1, X2 = 6
at the lower bounds the value of the objective is 1000
The distance from the values are delX1 = 0.0, delX2 = 0.16666666666666696


**Node 5**

Retain the constraint from node 3

$X_2 >= 6$

Add the constraint

$X_1 >=2$

In [100]:
node5 = declare()
node5.cons_fix = Constraint(expr = node4.X2 >= 6)
node5.cons_add = Constraint(expr = node5.X1 >= 2)
result = SolverFactory('gurobi').solve(node5)
print_res(node5)

KeyError: 4742152480

Infeasible, this node is lame

**Node 6**

Retain the constraints from Node 4 

$X_2 >= 6$

$X_1 <=1$

Add the constraint $X_2 <= 6$


In [97]:
node6 = declare()

node6.cons_fix = Constraint(expr = node6.X2 <= 6)
node6.cons_add = Constraint(expr = node6.X1 <= 1)
result = SolverFactory('gurobi').solve(node6)
print_res(node6)

objective value is 1000.0, X1 = 1.0, X2 = 6.0
The lower bounds of variables are X1 = 1, X2 = 6
at the lower bounds the value of the objective is 1000
The distance from the values are delX1 = 0.0, delX2 = 0.0


**Node 7**

Retain the constraints from Node 4 

$X_2 >= 6$

$X_1 <=1$

Add the constraint $X_2 >= 7$


In [99]:
node7 = declare()
node7.cons_fix = Constraint(expr = node7.X2 >= 7)
node7.cons_add = Constraint(expr = node7.X1 <= 1)
result = SolverFactory('gurobi').solve(node7)
print_res(node7)

    model.name="unknown";
      - termination condition: infeasibleOrUnbounded
      - message from solver: Problem proven to be infeasible or unbounded.
ERROR: evaluating object as numeric value: X1
        (object: <class 'pyomo.core.base.var.ScalarVar'>)
    No value for uninitialized NumericValue object X1


ValueError: No value for uninitialized NumericValue object X1

This node is lame

In [12]:
def declare():
    model = ConcreteModel()

    model.X1 = Var(within = NonNegativeIntegers)
    model.X2 = Var(within = NonNegativeIntegers)

    model.money_cons = Constraint(expr = 8000*model.X1 + 4000*model.X2 <= 40000)
    model.area_cons = Constraint(expr = 15*model.X1 + 30*model.X2 <= 200)

    model.obj = Objective(expr = 100*model.X1 + 150*model.X2, sense = maximize)

    return model

model = declare()

result = SolverFactory('gurobi').solve(model)
print_res(model)

objective value is 1000.0, X1 = 1.0, X2 = 6.0
The lower bounds of variables are X1 = 1, X2 = 6
at the lower bounds the value of the objective is 1000
The distance from the values are delX1 = 0.0, delX2 = 0.0


## Node 6 is king